In [1]:
#-*- coding:utf-8 -*-
from pyecharts.charts import Bar
from pyecharts import options as opts

bar = (
    Bar()
    .add_xaxis(["衬衫", "毛衣", "领带", "裤子", "风衣", "高跟鞋", "袜子"])
    .add_yaxis("商家A", [114, 55, 27, 101, 125, 27, 105])
    .add_yaxis("商家B", [57, 134, 137, 129, 145, 60, 49])
    .set_global_opts(title_opts=opts.TitleOpts(title="某商场销售情况"))
)
 
bar.render(r"\test1.html")

'D:\\test1.html'

In [91]:
import tushare as ts 
import pandas as pd
from datetime import datetime

#获取当前正常上市交易的股票列表
def get_code():
    pro = ts.pro_api()
    date = (datetime.now() + timedelta(days=-1)).strftime('%Y%m%d') #当前日期的前一天，转为字符串
    df_db= pro.query('daily_basic', ts_code='', trade_date=date,fields='ts_code,trade_date,\
        turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv')
    df_db['share_ratio'] = df_db.free_share / df_db.total_share * 100 #计算自由流通股占比
    df_db['total_mv'] = df_db.total_mv  / 10000 #把总市值单位转为亿元
    filt = (df_db['total_mv'] > 100 ) & (df_db['share_ratio'] > 20)  #总市值 > 100亿,流通股占比大于20%
    df_db =  df_db[filt] 
    c_1 = set(df_db.ts_code.to_list()) #取set
    
    df = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
    year = (datetime.now() +timedelta(days=-365*3)).strftime('%Y%m%d') #上市时间为三年前的今天
    #保留上市时间大于三年个股数据
    df=df[df.list_date<year]
    #排除银行、保险、多元金融公司
    df=df[-df.industry.isin(['银行','保险','多元金融'])]
    #排除st和*ST股
    df=df[-df.name.str.startswith(('ST'))]
    df=df[-df.name.str.startswith(('*'))]
    c_2 = set(df.ts_code.to_list())
    
    c = c_1 & c_2 #求交集
    df = df[df.ts_code.isin(c)]
    code=df.ts_code.values
    name=df.name.values
    return dict(zip(name,code))
    
    
get_code()
    
    

{'万科A': '000002.SZ',
 '中国宝安': '000009.SZ',
 '南玻A': '000012.SZ',
 '深康佳A': '000016.SZ',
 '深科技': '000021.SZ',
 '深圳能源': '000027.SZ',
 '国药一致': '000028.SZ',
 '富奥股份': '000030.SZ',
 '大悦城': '000031.SZ',
 '神州数码': '000034.SZ',
 '中国天楹': '000035.SZ',
 '中集集团': '000039.SZ',
 '德赛电池': '000049.SZ',
 '深天马A': '000050.SZ',
 '中金岭南': '000060.SZ',
 '农产品': '000061.SZ',
 '深圳华强': '000062.SZ',
 '中兴通讯': '000063.SZ',
 '中国长城': '000066.SZ',
 '华侨城A': '000069.SZ',
 '盐田港': '000088.SZ',
 '深圳机场': '000089.SZ',
 'TCL科技': '000100.SZ',
 '川能动力': '000155.SZ',
 '华数传媒': '000156.SZ',
 '中联重科': '000157.SZ',
 '常山北明': '000158.SZ',
 '申万宏源': '000166.SZ',
 '东方盛虹': '000301.SZ',
 '美的集团': '000333.SZ',
 '潍柴动力': '000338.SZ',
 '许继电气': '000400.SZ',
 '冀东水泥': '000401.SZ',
 '金融街': '000402.SZ',
 '派林生物': '000403.SZ',
 '东旭光电': '000413.SZ',
 '吉林化纤': '000420.SZ',
 '湖北宜化': '000422.SZ',
 '东阿阿胶': '000423.SZ',
 '徐工机械': '000425.SZ',
 '兴业矿业': '000426.SZ',
 '粤高速A': '000429.SZ',
 '晨鸣纸业': '000488.SZ',
 '丽珠集团': '000513.SZ',
 '国际医学': '000516.SZ',
 '中兵红箭': '000519

In [5]:
import tushare as ts 
import pandas as pd
from datetime import datetime

def get_indicators(code):
    '''
    {'营业收入同比增长率':'tr_yoy','营业利润同比增长率':'op_yoy','毛利率':'grossprofit_margin',
    '期间费用率':'expense_of_sales','存货周转率':'inv_turn','每股经营性现金流量':'ocfps', 
    '每股收益':'eps','加权净资产收益率':'roe_waa','年化净资产收益率':'roe_yearly', 
    '年化总资产报酬率':'roa2_yearly','归属母公司股东的净利润-扣除非经常损益同比增长率':'dt_netprofit_yoy'}
    '''
    pro = ts.pro_api()
    #获取当前时间，计算当前和过去四年时间
    t0=datetime.now()
    t1=datetime(t0.year-4,t0.month,t0.day)
    end=t0.strftime('%Y%m%d')
    start=t1.strftime('%Y%m%d')
    #财务比率
    fields='ann_date,end_date,tr_yoy,op_yoy,\
         grossprofit_margin,expense_of_sales,inv_turn,eps,\
         ocfps,roe_yearly,roa2_yearly,netprofit_yoy'
    fina = (pro.fina_indicator(ts_code=code,start_date=start, end_date=end,fields=fields)
           .drop_duplicates(subset=['ann_date','end_date'],keep='first'))
    fina.set_index('end_date',inplace = True)
    fina=fina.sort_index()
    #获取市盈率和市净率指标（pe、pb数据）
    pbe=pro.daily_basic(ts_code=code, fields='trade_date,pe_ttm,pb,close')
    pbe.set_index('trade_date',inplace=True)
    pbe=pbe.sort_index()
    #合并数据
    df=pd.merge(fina,pbe,left_index=True,right_index=True,how='left')
    #pb缺失数据使用前值填充，pe不管，缺失值可能是因为盈利为负数
    df['pb'].fillna(method='ffill',inplace=True)
    return df
get_indicators('600196.SH')



,ann_date,eps,inv_turn,ocfps,grossprofit_margin,expense_of_sales,roe_yearly,roa2_yearly,op_yoy,netprofit_yoy,tr_yoy,close,pe_ttm,pb
end_date,,,,,,,,,,,,,,
20171231,20180327,1.27,3.4420,1.0341,58.9450,49.0714,13.1511,8.6251,20.0496,11.3571,26.6921,NaN,NaN,NaN
20180331,20180428,0.28,0.8503,0.1868,57.5933,47.3260,11.0188,7.4360,1.3624,-4.3325,47.3849,NaN,NaN,NaN
20180630,20180828,0.63,1.6938,0.5125,58.2944,49.5435,12.2654,7.5496,-6.2209,-7.6130,41.9726,NaN,NaN,NaN
20180930,20181030,0.83,2.6146,0.8146,58.2727,50.5918,10.3691,6.8151,-10.6463,-13.4386,39.9896,NaN,NaN,NaN
20181231,20190326,1.07,3.4334,1.1510,58.4028,51.8927,10.1602,6.5853,-13.2362,-13.3326,34.4495,NaN,NaN,NaN
20190331,20190430,0.28,0.8059,0.1593,59.1234,51.3649,10.0504,6.7784,2.2990,0.9221,17.6740,NaN,NaN,NaN
20190630,20190827,0.59,1.6169,0.5658,60.4948,52.4458,10.6946,7.3666,8.7770,-2.8421,19.5107,NaN,NaN,NaN
20190930,20191030,0.81,2.3891,0.9085,59.7851,52.5302,9.3893,6.5097,5.4349,-1.4540,17.0057,25.27,24.3149,2.3206
20191231,20200331,1.30,3.1941,1.2573,59.6174,53.6795,11.0969,7.3822,27.0969,22.6629,14.7156,26.60,25.4617,2.2254


In [4]:
(((12 /9) * 1.39 + (12/6) * 0.97 + (12 /3) * 0.33 + 1.43) /4) * 30.254


49.49050166666667